In [ ]:
import pandas as pd
import numpy as np
pd.options.display.max_columns = None
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
# import matplotlib.style as style
# style.use('fivethirtyeight')
import warnings
warnings.filterwarnings('ignore')
from datetime import date
from dateutil.relativedelta import relativedelta
import dataframe_image as dfi
from adjustText import adjust_text

In [ ]:
%run operations_utils.py

In [ ]:
%run path_changer_refill.py

In [ ]:
df = pd.read_csv(source_file_path+'rxflow_mar2aug.csv')

In [ ]:
df.rename(columns = {'erxreceived': 'track_erx_received',
 'rx_entered_date': 'track_rx_entered',
 'datef': 'track_rx_filled',
 'rx_filledon_date':'filled_datekey',
 'rx_pickedup_date': 'final_rx_pickedup',
 'new_delivered_date':'track_rx_delivered',
 'rx_labelprint_date': 'track_label_printed',
 'rx_drugpick_date': 'final_rx_drug_picks',
 'rx_rphpick_date': 'final_rx_rph_picks',
 'first_billed_date': 'first_billed_datetime',
 'last_billed_date': 'end_billed_datetime',
 'processing_start': 'trackrx_processing_date_starttime',
 'patmobileno': 'PATMOBILENO',
 'patphone': 'PATPHONE',
 'totalrxamount': 'TOTALRXAMOUNT',
 'trackrx_last_status_tag': 'trackrx_last_status'}, inplace = True)

In [ ]:
# df.rename(columns = {
#  'erxreceived': 'track_erx_received',
#  'rx_entered_date': 'track_rx_entered',
#  'rx_filledon_date': 'filled_datekey',
#  'rx_pickedup_date': 'track_rx_pickedup',
#  'rx_delivered_date': 'track_rx_delivered',
#  'rx_labelprint_date': 'track_label_printed',
#  'rx_drugpick_date': 'final_rx_drug_picks',
#  'rx_rphpick_date': 'final_rx_rph_picks',
#  'first_billed_date': 'first_billed_datetime',
#  'last_billed_date': 'end_billed_datetime',
#  'processing_start': 'trackrx_processing_date_starttime',
#  'patmobileno': 'PATMOBILENO',
#  'patphone': 'PATPHONE',
#  'totalrxamount': 'TOTALRXAMOUNT',
#  'trackrx_last_status_tag': 'trackrx_last_status'}, inplace = True)

In [ ]:
df = df[~df['rx_code_key_location'].str.endswith('-0', na = False)]

In [ ]:
df.head()

In [ ]:
df['track_rx_recieved_new'] =  pd.to_datetime(df.track_erx_received, dayfirst = False)
df['track_rx_entered_new'] =  pd.to_datetime(df.track_rx_entered, dayfirst = False)
df['filled_datekey'] =  pd.to_datetime(df.filled_datekey, dayfirst = False)

# df['max_date'] = df[['track_rx_recieved_new','track_rx_entered_new']].max(axis=1)
# df['max_date_only'] = df['max_date'].dt.date
df['pharmacy_loc'] = df['rx_code_key_location'].astype(str).str[0:5]
df['mon_key'] = df['filled_datekey'].dt.strftime('%m-%y')
df['mon_key_2'] = df['filled_datekey'].dt.strftime('%b-%y')
# df = df[df['mon_key_2']!='Jan-21']

In [ ]:
mon = ['Mar-21','Apr-21','May-21','Jun-21','Jul-21','Aug-21']
df = df[df['mon_key_2'].isin(mon)]
df['mon_key'].min()

In [ ]:
# six_months = date.today() - relativedelta(months=+6)
# six_months = six_months.strftime("%Y-%m")

# six_month_date = str(six_months)+"-01"
# six_month_date

In [ ]:
# five_months = date.today() - relativedelta(months=+5)
# five_months = five_months.strftime("%Y-%m")

# five_month_date = str(five_months)+"-01"
# five_month_date

In [ ]:
# four_months = date.today() - relativedelta(months=+4)
# four_months

In [ ]:
df['trackrx_last_status'].unique()
df['trackrx_last_status_new'] = df['trackrx_last_status']
df['trackrx_last_status_new'] = df['trackrx_last_status'].fillna('B')
df['trackrx_last_status_new'] = df['trackrx_last_status'].replace(['F', 'U', 'T'], 'NB')

In [ ]:
df.is_pickedup.value_counts()

In [ ]:
# df.status.value_counts()

In [ ]:
'status' in df.columns

In [ ]:
conditions = [
    (df['is_pickedup'] == False) & (df['status'] == 'completed'),
    (df['is_pickedup'] == True)
    ]
# , del_data['status'] == 'NaN'
# create a list of the values we want to assign for each condition
values = ['Delivered', 'Pickedup']

# create a new column and use np.select to assign values to it using our lists as arguments
df['delivery_status'] = np.select(conditions, values)
df['delivery_status'].replace('0', np.nan, inplace = True)
# display updated DataFrame

In [ ]:
df.delivery_status.value_counts()

In [ ]:
df = df.rename(columns = {'pharmacy_loc': 'pharmacy loc'})


In [ ]:
df = df.loc[(df['pharmacy loc'] != 'TX-DA') & (df['pharmacy loc'] != 'TX-SA') & (df['pharmacy loc'] != 'GA-AT') & (df['pharmacy loc'] != 'NY-UC')& (df['pharmacy loc'] != 'CT-NH')]
df['pharmacy loc'].unique()

In [ ]:
dff = df.copy()

In [ ]:
erx_data_overall = pd.pivot_table(df, values = 'rx_code_key_location', columns = 'is_eRx', index = 'trackrx_last_status_new', aggfunc = 'count').reset_index()

In [ ]:
erx_data_overall.drop(index=2,inplace=True)
erx_data_overall

In [ ]:
erx_data_overall = pd.pivot_table(df, values = 'rx_code_key_location', columns = 'is_eRx', index = 'trackrx_last_status_new', aggfunc = 'count').reset_index()
erx_data_overall[True] = 0
erx_data_overall['Total'] = erx_data_overall[False] + erx_data_overall[True]

erx_data_overall['not eRx'] = round((erx_data_overall[False]/erx_data_overall['Total'])*100,2)
erx_data_overall['eRx'] = round((erx_data_overall[True]/erx_data_overall['Total'])*100,2)
erx_data_plot = erx_data_overall[['trackrx_last_status_new', 'not eRx', 'eRx']]
erx_data_plot.set_index('trackrx_last_status_new', inplace = True)
erx_data_plot.drop('None',inplace=True)
erx_data_plot

In [ ]:
ax = erx_data_plot.plot(kind = "bar", rot = 360)

list_values = [erx_data_overall[False][0], erx_data_overall[False][1],erx_data_overall[True][0], erx_data_overall[True][1]]

for p,k in zip(ax.patches, list_values):
    width, height = p.get_width(), p.get_height()
    x, y = p.get_xy() 
    ax.text(x+width/2, y+height/2, str(int(height))+'%', horizontalalignment='center', verticalalignment='center')
    ax.text(x+width/2, y+height+2, f'{int(k):,}', horizontalalignment='center', verticalalignment='center')
    
plt.ylabel("% of Rx",fontsize = 16,fontname = 'Arial')
plt.xlabel("Rx status",fontsize = 16,fontname = 'Arial')
plt.title("Mode of transfer by billing status",fontsize = 16,fontname = 'Arial')

plt.xticks((0,1), ('Billed', 'Non Billed'))

plt.legend(title='is eRx?', bbox_to_anchor=(1.0, 1), loc='upper left')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)
# plt.show()
plt.savefig(output_image_path_billed+'Mode_of_transfer_by_billing_status.png', bbox_inches='tight', dpi=300,pad_inches = 0)
plt.close()


In [ ]:
erx_data = df[['rx_code_key_location', 'is_eRx','mon_key_2', 'trackrx_last_status_new', 'pharmacy loc', 'is_pickedup','delivery_status']]

In [ ]:
erx_data

In [ ]:
# erx_data1 = erx_data[(erx_data['max_date'] > six_month_date)]
# erx_data1.sort_values(by= 'max_date')

In [ ]:
erx_data1 = erx_data

In [ ]:
eRx_month_wise = pd.pivot_table(erx_data1, values = 'rx_code_key_location', columns = 'is_eRx', index = ['mon_key_2'], aggfunc = 'count').reset_index()
eRx_month_wise[True]=0
eRx_month_wise['mon_key_2'] = pd.to_datetime(eRx_month_wise['mon_key_2'], format = "%b-%y")
eRx_month_wise['mon_key_3'] = eRx_month_wise['mon_key_2'].dt.strftime('%b-%y')
eRx_month_wise.sort_values(by = 'mon_key_2', inplace = True)
eRx_month_wise.drop(columns = 'mon_key_2', inplace = True)
eRx_month_wise.rename(columns = {'mon_key_3':'mon_key_2'}, inplace = True)
eRx_month_wise

In [ ]:
eRx_month_wise_line = eRx_month_wise
eRx_month_wise_line.columns
eRx_month_wise_line['Total'] = eRx_month_wise[False] + eRx_month_wise[True]
eRx_month_wise_line['Per change'] = round(eRx_month_wise['Total'].pct_change()*100,2)
eRx_month_wise_line = eRx_month_wise_line.fillna(0)

eRx_month_wise_line

In [ ]:
eRx_month_wise = pd.pivot_table(erx_data1, values = 'rx_code_key_location', columns = 'is_eRx', index = ['mon_key_2'], aggfunc = 'count').reset_index()
eRx_month_wise[True]=0
eRx_month_wise['mon_key_2'] = pd.to_datetime(eRx_month_wise['mon_key_2'], format = "%b-%y")
eRx_month_wise['mon_key_3'] = eRx_month_wise['mon_key_2'].dt.strftime('%b-%y')
eRx_month_wise.sort_values(by = 'mon_key_2', inplace = True)
eRx_month_wise.drop(columns = 'mon_key_2', inplace = True)
eRx_month_wise.rename(columns = {'mon_key_3':'mon_key_2'}, inplace = True)
eRx_month_wise

In [ ]:
ax = eRx_month_wise.plot(kind = 'bar', stacked = True, x = 'mon_key_2', rot = 360, figsize = (10,5), legend = False)
eRx_month_wise_line.plot(ax = ax, kind = 'line', x = 'mon_key_2', y = 'Total', color = 'grey', legend = False)


for p in ax.patches:
    width, height = p.get_width(), p.get_height()
    x, y = p.get_xy() 
    ax.text(x+width/2, y+height/2, f'{int(height):,}', horizontalalignment='center', verticalalignment='center')
    

rects = ax.patches

texts=[]

for rect, total, per in zip(rects, eRx_month_wise_line['Total'], eRx_month_wise_line['Per change']):
    text = ax.text(rect.get_x() + rect.get_width() / 2, total, f'{int(total):,}'+' '+ '(' + str(per) +'%'+')',
            ha='center', va='center')
    texts.append(text)
adjust_text(texts)

max_y_limit = eRx_month_wise_line['Total'].max()
per_10 = (10/100) * max_y_limit + max_y_limit
ax.set_ylim(0, per_10)

plt.ylabel("Count of Rx",fontsize = 16,fontname = 'Arial')
plt.xlabel("Month Year",fontsize = 16,fontname = 'Arial')
plt.title("Overall Rx M-o-M analysis",fontsize = 16,fontname = 'Arial')

red_patch = mpatches.Patch(color='orange', label='eRx')
blue_patch = mpatches.Patch(color='steelblue', label='not eRx')
trend_patch = mpatches.Patch(color='grey', label='Total')
plt.legend(handles=[red_patch, blue_patch, trend_patch], title='is eRx?', bbox_to_anchor=(1.0, 1), loc='upper left')

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)
#plt.legend(title='is eRx?', bbox_to_anchor=(1.0, 1), loc='upper left')

plt.savefig(output_image_path_billed+'Overall_M-o-M_analysis_Rx_count.png', bbox_inches='tight', dpi=300,pad_inches = 0)
plt.close()

In [ ]:
eRx_pharmacy_wise = pd.pivot_table(erx_data1, values = 'rx_code_key_location', columns = 'is_eRx', index = ['pharmacy loc'], aggfunc = 'count').reset_index()
eRx_pharmacy_wise.rename(columns = {False: 'False', True: 'True'}, inplace =True)
eRx_pharmacy_wise['True']=0

df_total = eRx_pharmacy_wise['False'] + eRx_pharmacy_wise['True']
eRx_pharmacy_wise['False'] = eRx_pharmacy_wise[eRx_pharmacy_wise.columns[1:2]].div(df_total, 0)*100
eRx_pharmacy_wise['True'] = eRx_pharmacy_wise[eRx_pharmacy_wise.columns[2:]].div(df_total, 0)*100

ax = eRx_pharmacy_wise.plot(kind = 'bar', stacked = True, x = 'pharmacy loc', rot = 360, legend = False)

for p in ax.patches:
    width, height = p.get_width(), p.get_height()
    x, y = p.get_xy() 
    ax.text(x+width/2, y+height/2, '{:.0f}%'.format(height), horizontalalignment='center', verticalalignment='center')

plt.ylabel("% of Rx",fontsize = 16,fontname = 'Arial')
plt.xlabel("City Location",fontsize = 16,fontname = 'Arial')
plt.title("Mode of Rx Transfer Overall",fontsize = 16,fontname = 'Arial')

#plt.legend(title='is eRx?', bbox_to_anchor=(1.0, 1), loc='upper left')
orange_patch = mpatches.Patch(color='orange', label='eRx')
blue_patch = mpatches.Patch(color='steelblue', label='not eRx')
plt.legend(handles=[orange_patch, blue_patch], title='is eRx?', bbox_to_anchor=(1.0, 1), loc='upper left')
#ax.axis('Off')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)

plt.savefig(output_image_path_billed+'Mode_of_Rx_transfer_Overall.png', bbox_inches='tight', dpi=300,pad_inches = 0)
plt.close()

In [ ]:
overall_pharmacy_wise_Rx = pd.pivot_table(erx_data1, values = 'rx_code_key_location', columns = 'pharmacy loc', index = ['is_eRx'], aggfunc = 'count').reset_index()
# overall_pharmacy_wise_Rx = overall_pharmacy_wise_Rx.append(overall_pharmacy_wise_Rx.sum(axis=0), ignore_index = True)
overall_pharmacy_wise_Rx = overall_pharmacy_wise_Rx.append(pd.Series(), ignore_index = True)
overall_pharmacy_wise_Rx = overall_pharmacy_wise_Rx.fillna(0)
overall_pharmacy_wise_Rx = overall_pharmacy_wise_Rx.append(overall_pharmacy_wise_Rx.sum(axis=0), ignore_index = True)
# overall_pharmacy_wise_Rx['Total'] = overall_pharmacy_wise_Rx.sum(axis=1)


overall_pharmacy_wise_Rx = overall_pharmacy_wise_Rx.rename(columns = {'is_eRx': 'pharmacy loc/is eRx'})
overall_pharmacy_wise_Rx.iloc[[0],[0]] = 'not eRx'
overall_pharmacy_wise_Rx.iloc[[1],[0]] = 'is eRx'
overall_pharmacy_wise_Rx.iloc[[2],[0]] = 'Total'
overall_pharmacy_wise_Rx.set_index('pharmacy loc/is eRx', inplace = True)
# #np.argsort(overall_pharmacy_wise_Rx, axis=0)
overall_pharmacy_wise_Rx['Total'] = overall_pharmacy_wise_Rx.sum(axis=1)
overall_pharmacy_wise_Rx = overall_pharmacy_wise_Rx[overall_pharmacy_wise_Rx.columns[:]].div(6, 0).astype(int)
overall_pharmacy_wise_Rx = overall_pharmacy_wise_Rx.sort_values(overall_pharmacy_wise_Rx.last_valid_index(), axis=1, ascending = False)
overall_pharmacy_wise_Rx = overall_pharmacy_wise_Rx[['NY-BK', 'NJ-SO', 'PA-PH', 'FL-MI', 'NY-MA', 'MD-BA', 'NC-RA','Total']]

subset_value = (overall_pharmacy_wise_Rx.index.to_list()[:-1], 
               ['NY-BK', 'NJ-SO', 'PA-PH', 'FL-MI', 'NY-MA', 'MD-BA', 'NC-RA'])

df_style = overall_pharmacy_wise_Rx.style.set_properties(**{'text-align': 'center','border-color':'white','border-width':'thin','border-style':'solid'}).background_gradient(cmap='GnBu',axis=1, subset = subset_value).format("{:,}")

dfi.export(df_style, output_image_path_billed+'overall_pharmacy_wise_Rx_count.png')
df_style


In [ ]:
# Billed

In [ ]:
erx_data_billed = erx_data[erx_data['trackrx_last_status_new'] == 'B']
#erx_data_billed
# erx_data_billed1 = erx_data_billed[(erx_data_billed['max_date'] > six_month_date)]
#erx_data_billed1

In [ ]:
erx_data_billed1 = erx_data_billed

In [ ]:
billed_pie_pivot = (pd.pivot_table(erx_data_billed1, values = 'rx_code_key_location', index=['is_eRx'], aggfunc = 'count')/6).reset_index()
billed_pie_pivot = billed_pie_pivot.append(pd.Series(),ignore_index=True)
billed_pie_pivot.iloc[[1],[0]] = 1
billed_pie_pivot.iloc[[1],[1]] = 0
billed_pie_pivot

In [ ]:
mylabels = (billed_pie_pivot['rx_code_key_location']/6).to_list()
my_labels = 'not eRx '+str(f'{int(mylabels[0]):,}'),'eRx '+str(f'{int(mylabels[1]):,}')
my_explode = (0, 0.1)
plt.pie(billed_pie_pivot['rx_code_key_location']/6, labels=my_labels, autopct='%1.2f%%', startangle=90, shadow = False, explode=my_explode)
plt.title('Mode of Rx Transfer Billed')
plt.axis('equal')

plt.savefig(output_image_path_billed+'Mode_of_Rx_Transfer_Billed_pie.png', bbox_inches='tight', dpi=300,pad_inches = 0)
plt.close()

In [ ]:
billed_eRx_month_wise = pd.pivot_table(erx_data_billed1, values = 'rx_code_key_location', columns = 'is_eRx', index = ['mon_key_2'], aggfunc = 'count').reset_index()

billed_eRx_month_wise['mon_key_2'] = pd.to_datetime(billed_eRx_month_wise['mon_key_2'], format = "%b-%y")
billed_eRx_month_wise['mon_key_3'] = billed_eRx_month_wise['mon_key_2'].dt.strftime('%b-%y')
billed_eRx_month_wise.sort_values(by = 'mon_key_2', inplace = True)
billed_eRx_month_wise.drop(columns = 'mon_key_2', inplace = True)
billed_eRx_month_wise.rename(columns = {'mon_key_3':'mon_key_2'}, inplace = True)
billed_eRx_month_wise

In [ ]:
billed_eRx_month_wise_line = billed_eRx_month_wise
billed_eRx_month_wise_line.columns
billed_eRx_month_wise[True] = 0
billed_eRx_month_wise_line['Total'] = billed_eRx_month_wise[False] + billed_eRx_month_wise[True]
billed_eRx_month_wise_line['per change'] = round(billed_eRx_month_wise['Total'].pct_change()*100,2)
billed_eRx_month_wise_line = billed_eRx_month_wise_line.fillna(0)
billed_eRx_month_wise_line

In [ ]:
billed_eRx_month_wise = pd.pivot_table(erx_data_billed1, values = 'rx_code_key_location', columns = 'is_eRx', index = ['mon_key_2'], aggfunc = 'count').reset_index()

billed_eRx_month_wise['mon_key_2'] = pd.to_datetime(billed_eRx_month_wise['mon_key_2'], format = "%b-%y")
billed_eRx_month_wise['mon_key_3'] = billed_eRx_month_wise['mon_key_2'].dt.strftime('%b-%y')
billed_eRx_month_wise.sort_values(by = 'mon_key_2', inplace = True)
billed_eRx_month_wise.drop(columns = 'mon_key_2', inplace = True)
billed_eRx_month_wise.rename(columns = {'mon_key_3':'mon_key_2'}, inplace = True)
#billed_eRx_month_wise

In [ ]:
ax = billed_eRx_month_wise.plot(kind = 'bar', stacked = True, x = 'mon_key_2', rot = 360, figsize = (10,5), legend = False)
billed_eRx_month_wise_line.plot(ax = ax, kind = 'line', x = 'mon_key_2', y = 'Total', color = 'grey', legend = False)

for p in ax.patches:
    width, height = p.get_width(), p.get_height()
    x, y = p.get_xy() 
    ax.text(x+width/2, y+height/2, f'{int(height):,}', horizontalalignment='center', verticalalignment='center')

rects = ax.patches

texts = []

for rect, total, per in zip(rects, billed_eRx_month_wise_line['Total'], billed_eRx_month_wise_line['per change']):
    text = ax.text(rect.get_x() + rect.get_width() / 2, total, f'{int(total):,}'+' '+ '(' + str(per) +'%'+')',
            ha='center', va='center')
    texts.append(text)
adjust_text(texts)
plt.ylabel("Count of Rx",fontsize = 16,fontname = 'Arial')
plt.xlabel("Month Year",fontsize = 16,fontname = 'Arial')
plt.title("Billed Rx M-o-M analysis",fontsize = 16,fontname = 'Arial')

max_y_limit = billed_eRx_month_wise_line['Total'].max()
per_10 = (10/100) * max_y_limit + max_y_limit
ax.set_ylim(0, per_10)

orange_patch = mpatches.Patch(color='orange', label='eRx')
blue_patch = mpatches.Patch(color='steelblue', label='not eRx')
trend_patch = mpatches.Patch(color='grey', label='Total')
plt.legend(handles=[orange_patch, blue_patch, trend_patch], title='is eRx?', bbox_to_anchor=(1.0, 1), loc='upper left')

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)
#plt.legend(title='is eRx?', bbox_to_anchor=(1.0, 1), loc='upper left')

plt.savefig(output_image_path_billed+'billed_M-o-M_analysis_eRx.png', bbox_inches='tight', dpi=300,pad_inches = 0)
plt.close()

In [ ]:
billed_eRx_pharmacy_wise = pd.pivot_table(erx_data_billed1, values = 'rx_code_key_location', columns = 'is_eRx', index = ['pharmacy loc'], aggfunc = 'count').reset_index()
billed_eRx_pharmacy_wise.rename(columns = {False: 'False', True: 'True'}, inplace =True)
billed_eRx_pharmacy_wise['True'] = 0
df_total = billed_eRx_pharmacy_wise['False'] + billed_eRx_pharmacy_wise['True']
billed_eRx_pharmacy_wise['False'] = billed_eRx_pharmacy_wise[billed_eRx_pharmacy_wise.columns[1:2]].div(df_total, 0)*100
billed_eRx_pharmacy_wise['True'] = billed_eRx_pharmacy_wise[billed_eRx_pharmacy_wise.columns[2:]].div(df_total, 0)*100

ax = billed_eRx_pharmacy_wise.plot(kind = 'bar', stacked = True, x = 'pharmacy loc', rot = 360, legend = False)

for p in ax.patches:
    width, height = p.get_width(), p.get_height()
    x, y = p.get_xy() 
    ax.text(x+width/2, y+height/2, '{:.0f}%'.format(height), horizontalalignment='center', verticalalignment='center')


plt.ylabel("Percentage Rx",fontsize = 16,fontname = 'Arial')
plt.xlabel("Pharmacy Location",fontsize = 16,fontname = 'Arial')
plt.title("Location wise mode of Rx Transfer",fontsize = 16,fontname = 'Arial')

#plt.legend(title='is eRx?', bbox_to_anchor=(1.0, 1), loc='upper left')
orange_patch = mpatches.Patch(color='orange', label='eRx')
blue_patch = mpatches.Patch(color='steelblue', label='not eRx')
plt.legend(handles=[orange_patch, blue_patch], title='is eRx?', bbox_to_anchor=(1.0, 1), loc='upper left')

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)

plt.savefig(output_image_path_billed+'Location_wise_mode_of_Rx_Transfer_billed.png', bbox_inches='tight', dpi=300,pad_inches = 0)
plt.close()


In [ ]:
Billed_pharmacy_wise_Rx = pd.pivot_table(erx_data_billed1, values = 'rx_code_key_location', columns = 'pharmacy loc', index = ['is_eRx'], aggfunc = 'count').reset_index()
#Billed_pharmacy_wise_Rx = Billed_pharmacy_wise_Rx.append(Billed_pharmacy_wise_Rx.sum(axis=0), ignore_index = True)
Billed_pharmacy_wise_Rx = Billed_pharmacy_wise_Rx.append(pd.Series(), ignore_index = True)

# Billed_pharmacy_wise_Rx['Total'] = Billed_pharmacy_wise_Rx.sum(axis=1)


Billed_pharmacy_wise_Rx = Billed_pharmacy_wise_Rx.rename(columns = {'is_eRx': 'pharmacy loc/is eRx'})
Billed_pharmacy_wise_Rx.iloc[[0],[0]] = 'not eRx'
Billed_pharmacy_wise_Rx.iloc[[1],[0]] = 'is eRx'
Billed_pharmacy_wise_Rx = Billed_pharmacy_wise_Rx.fillna(0)
Billed_pharmacy_wise_Rx = Billed_pharmacy_wise_Rx.append(Billed_pharmacy_wise_Rx.sum(axis=0), ignore_index = True)

Billed_pharmacy_wise_Rx.iloc[[2],[0]] = 'Total'
Billed_pharmacy_wise_Rx.set_index('pharmacy loc/is eRx', inplace = True)
Billed_pharmacy_wise_Rx['Total'] = Billed_pharmacy_wise_Rx.sum(axis=1)
Billed_pharmacy_wise_Rx = Billed_pharmacy_wise_Rx[Billed_pharmacy_wise_Rx.columns[:]].div(6,0).astype(int)
Billed_pharmacy_wise_Rx = Billed_pharmacy_wise_Rx.sort_values(Billed_pharmacy_wise_Rx.last_valid_index(), axis=1, ascending = False)

Billed_pharmacy_wise_Rx = Billed_pharmacy_wise_Rx[['NY-BK', 'NJ-SO', 'PA-PH', 'FL-MI', 'NY-MA', 'MD-BA', 'NC-RA','Total']]

subset_value = (Billed_pharmacy_wise_Rx.index.to_list()[:-1], 
               ['NY-BK', 'NJ-SO', 'PA-PH', 'FL-MI', 'NY-MA', 'MD-BA', 'NC-RA'])

df_style = Billed_pharmacy_wise_Rx.style.set_properties(**{'text-align': 'center','border-color':'white','border-width':'thin','border-style':'solid'}).background_gradient(cmap='GnBu',axis=1, subset = subset_value).format("{:,}")


dfi.export(df_style, output_image_path_billed+'Billed_pharmacy_wise_Rx_count_table.png')
df_style

In [ ]:
# month wise analysis for city billed rx count

In [ ]:
# FL_MI

In [ ]:
# FL_MI_df = erx_data1[erx_data1['pharmacy loc'] == 'FL-MI']
# FL_MI_df_B = FL_MI_df[FL_MI_df['trackrx_last_status_new'] == 'B']
# FL_MI_df_B = FL_MI_df_B.loc[(FL_MI_df_B['max_date'] > six_month_date)]

# # FL_MI_df_B_eRx = FL_MI_df_B[FL_MI_df_B['is_eRx'] == True]
# FL_MI_df_B_non_eRx = FL_MI_df_B[FL_MI_df_B['is_eRx'] == False]

# FL_MI_eRx_month_wise_eRx = pd.pivot_table(FL_MI_df_B_non_eRx, values = 'rx_code_key_location', columns = 'is_eRx', index = ['mon_key_2'], aggfunc = 'count').reset_index()
# FL_MI_eRx_month_wise_eRx['mon_key_2'] = pd.to_datetime(FL_MI_eRx_month_wise_eRx['mon_key_2'], format = "%b-%y")
# FL_MI_eRx_month_wise_eRx['mon_key_3'] = FL_MI_eRx_month_wise_eRx['mon_key_2'].dt.strftime('%b-%y')
# FL_MI_eRx_month_wise_eRx.sort_values(by = 'mon_key_2', inplace = True)
# FL_MI_eRx_month_wise_eRx.drop(columns = 'mon_key_2', inplace = True)
# FL_MI_eRx_month_wise_eRx.rename(columns = {'mon_key_3': 'mon_key_2'}, inplace =True)

# FL_MI_eRx_month_wise_eRx['per_growth'] = round(FL_MI_eRx_month_wise_eRx[False].pct_change()*100,2)
# #FL_MI_eRx_month_wise_eRx
# avgdf = pd.DataFrame(columns = [False, 'mon_key_2'], index=range(0,1))
# avgdf[False] = int(FL_MI_eRx_month_wise_eRx[False].mean())
# avgdf['mon_key_2'] = 'avg'

# FL_MI_eRx_month_wise_eRx = FL_MI_eRx_month_wise_eRx.append(avgdf)
# FL_MI_eRx_month_wise_eRx = FL_MI_eRx_month_wise_eRx.fillna(0)
# print(FL_MI_eRx_month_wise_eRx)
# FL_MI_eRx_month_wise_line = FL_MI_eRx_month_wise_eRx

# if len(FL_MI_eRx_month_wise_line) == 5:
#     c = ['C0', 'C0', 'C0', 'C0', 'green']
# else:
#     c = ['C0', 'C0', 'C0', 'C0', 'C0', 'green']

# ax = FL_MI_eRx_month_wise_eRx.plot(kind = 'bar', x = 'mon_key_2', y = False, rot = 360, figsize = (10,5), legend = False, color=c)

# for p in ax.patches:
#     width, height = p.get_width(), p.get_height()
#     x, y = p.get_xy() 
#     ax.text(x+width/2, y+height/2, '{:.0f}'.format(height), horizontalalignment='center', verticalalignment='center', color = 'white')

# ax2 = FL_MI_eRx_month_wise_line[:-1].plot(ax = ax, secondary_y=True, kind = 'line', x = 'mon_key_2', y = 'per_growth', color = 'grey', legend = False)    

# texts = []

# for p,k in zip(ax.patches, FL_MI_eRx_month_wise_line.per_growth):
#     width, height = p.get_width(), p.get_height()
#     x, y = p.get_xy() 
#     text = ax.text(x+width/2, y+height, str(round(k,2))+'%', horizontalalignment='center', verticalalignment='center')
#     texts.append(text)
# adjust_text(texts)
# ax.set_xlabel("Month Year",fontsize = 16,fontname = 'Arial')
# ax.set_ylabel('Count of Rx', fontsize = 16,fontname = 'Arial')
# ax2.set_ylabel('per_growth', fontsize = 16,fontname = 'Arial')
# #ax.set_ylim(0,1400)

# max_y_limit = FL_MI_eRx_month_wise_line[False].max()
# per_10 = (10/100) * max_y_limit + max_y_limit
# ax.set_ylim(0, per_10)

# plt.title("eRx M-o-M analysis Trend",fontsize = 16,fontname = 'Arial')

# plt.legend(title='eRx', bbox_to_anchor=(1.0, 1), loc='upper left')

# ax2.spines['top'].set_visible(False)
# ax2.spines['left'].set_visible(False)
# ax2.spines['right'].set_visible(False)
# ax2.spines['bottom'].set_visible(False)

# ax.spines['top'].set_visible(False)
# ax.spines['left'].set_visible(False)
# ax.spines['right'].set_visible(False)
# ax.spines['bottom'].set_visible(False)

In [ ]:
# month wise analysis for city billed non rx count

In [ ]:
# FL_MI_df = erx_data1[erx_data1['pharmacy loc'] == 'FL-MI']
# FL_MI_df_B = FL_MI_df[FL_MI_df['trackrx_last_status_new'] == 'B']
# # FL_MI_df_B = FL_MI_df_B.loc[(FL_MI_df_B['max_date'] > six_month_date)]

# #FL_MI_df_B_eRx = FL_MI_df_B[FL_MI_df_B['is_eRx'] == True]
# FL_MI_df_B_non_eRx = FL_MI_df_B[FL_MI_df_B['is_eRx'] == False]

# FL_MI_eRx_month_wise_eRx = pd.pivot_table(FL_MI_df_B_non_eRx, values = 'rx_code_key_location', columns = 'is_eRx', index = ['mon_key_2'], aggfunc = 'count').reset_index()
# FL_MI_eRx_month_wise_eRx['mon_key_2'] = pd.to_datetime(FL_MI_eRx_month_wise_eRx['mon_key_2'], format = "%b-%y")
# FL_MI_eRx_month_wise_eRx['mon_key_3'] = FL_MI_eRx_month_wise_eRx['mon_key_2'].dt.strftime('%b-%y')
# FL_MI_eRx_month_wise_eRx.sort_values(by = 'mon_key_2', inplace = True)
# FL_MI_eRx_month_wise_eRx.drop(columns = 'mon_key_2', inplace = True)
# FL_MI_eRx_month_wise_eRx.rename(columns = {'mon_key_3': 'mon_key_2'}, inplace =True)

# FL_MI_eRx_month_wise_eRx['per_growth'] = round(FL_MI_eRx_month_wise_eRx[False].pct_change()*100,2)
# #FL_MI_eRx_month_wise_eRx
# avgdf = pd.DataFrame(columns = [False, 'mon_key_2'], index=range(0,1))
# avgdf[False] = int(FL_MI_eRx_month_wise_eRx[False].mean())
# avgdf['mon_key_2'] = 'avg'

# FL_MI_eRx_month_wise_eRx = FL_MI_eRx_month_wise_eRx.append(avgdf)
# FL_MI_eRx_month_wise_eRx = FL_MI_eRx_month_wise_eRx.fillna(0)
# print(FL_MI_eRx_month_wise_eRx)
# FL_MI_eRx_month_wise_line = FL_MI_eRx_month_wise_eRx

# if len(FL_MI_eRx_month_wise_eRx) == 5:
#     c = ['C0', 'C0', 'C0', 'C0', 'green']
# else:
#     c = ['C0', 'C0', 'C0', 'C0', 'C0', 'green']

# ax = FL_MI_eRx_month_wise_eRx.plot(kind = 'bar', x = 'mon_key_2', y = False, rot = 360, figsize = (10,5), legend = False, color=c)

# for p in ax.patches:
#     width, height = p.get_width(), p.get_height()
#     x, y = p.get_xy() 
#     ax.text(x+width/2, y+height/2, '{:.0f}'.format(height), horizontalalignment='center', verticalalignment='center', color = 'white')

# ax2 = FL_MI_eRx_month_wise_line[:-1].plot(ax = ax, secondary_y=True, kind = 'line', x = 'mon_key_2', y = 'per_growth', color = 'grey', legend = False)

# texts = []

# for p,k in zip(ax.patches, FL_MI_eRx_month_wise_line.per_growth):
#     width, height = p.get_width(), p.get_height()
#     x, y = p.get_xy() 
#     text = ax.text(x+width/2, y+height+22, str(round(k,2))+'%', horizontalalignment='center', verticalalignment='center')
#     texts.append(text)
# adjust_text(texts)
# ax.set_xlabel("Month Year",fontsize = 16,fontname = 'Arial')
# ax.set_ylabel('Count of Rx', fontsize = 16,fontname = 'Arial')
# ax2.set_ylabel('per_growth', fontsize = 16,fontname = 'Arial')
# #ax.set_ylim(0,1400)

# max_y_limit = FL_MI_eRx_month_wise_line[False].max()
# per_10 = (10/100) * max_y_limit + max_y_limit
# ax.set_ylim(0, per_10)

# plt.title("non eRx M-o-M analysis Trend",fontsize = 16,fontname = 'Arial')

# plt.legend(title='non eRx', bbox_to_anchor=(1.0, 1), loc='upper left')

# ax2.spines['top'].set_visible(False)
# ax2.spines['left'].set_visible(False)
# ax2.spines['right'].set_visible(False)
# ax2.spines['bottom'].set_visible(False)

# ax.spines['top'].set_visible(False)
# ax.spines['left'].set_visible(False)
# ax.spines['right'].set_visible(False)
# ax.spines['bottom'].set_visible(False)

In [ ]:
for i in np.unique(erx_data1['pharmacy loc']):
    FL_MI_df = erx_data1[erx_data1['pharmacy loc'] == i]
    FL_MI_df_B = FL_MI_df[FL_MI_df['trackrx_last_status_new'] == 'B']
#     FL_MI_df_B = FL_MI_df_B.loc[(FL_MI_df_B['max_date'] > six_month_date)]

    #FL_MI_df_B_eRx = FL_MI_df_B[FL_MI_df_B['is_eRx'] == True]
    FL_MI_df_B_non_eRx = FL_MI_df_B[FL_MI_df_B['is_eRx'] == False]

    FL_MI_eRx_month_wise_eRx = pd.pivot_table(FL_MI_df_B_non_eRx, values = 'rx_code_key_location', columns = 'is_eRx', index = ['mon_key_2'], aggfunc = 'count').reset_index()
    FL_MI_eRx_month_wise_eRx['mon_key_2'] = pd.to_datetime(FL_MI_eRx_month_wise_eRx['mon_key_2'], format = "%b-%y")
    FL_MI_eRx_month_wise_eRx['mon_key_3'] = FL_MI_eRx_month_wise_eRx['mon_key_2'].dt.strftime('%b-%y')
    FL_MI_eRx_month_wise_eRx.sort_values(by = 'mon_key_2', inplace = True)
    FL_MI_eRx_month_wise_eRx.drop(columns = 'mon_key_2', inplace = True)
    FL_MI_eRx_month_wise_eRx.rename(columns = {'mon_key_3': 'mon_key_2'}, inplace =True)

    FL_MI_eRx_month_wise_eRx['per_growth'] = round(FL_MI_eRx_month_wise_eRx[False].pct_change()*100,2)
    #FL_MI_eRx_month_wise_eRx
    avgdf = pd.DataFrame(columns = [False, 'mon_key_2'], index=range(0,1))
    avgdf[False] = int(FL_MI_eRx_month_wise_eRx[False].mean())
    avgdf['mon_key_2'] = 'avg'

    FL_MI_eRx_month_wise_eRx = FL_MI_eRx_month_wise_eRx.append(avgdf)
    FL_MI_eRx_month_wise_eRx = FL_MI_eRx_month_wise_eRx.fillna(0)
    FL_MI_eRx_month_wise_line = FL_MI_eRx_month_wise_eRx
    
    if len(FL_MI_eRx_month_wise_eRx) == 5:
        c = ['C0', 'C0', 'C0', 'C0', 'green']
    elif len(FL_MI_eRx_month_wise_eRx) == 6:
        c = ['C0', 'C0', 'C0', 'C0','C0', 'green']
    else:
        c = ['C0', 'C0', 'C0', 'C0', 'C0','C0', 'green']

    ax = FL_MI_eRx_month_wise_eRx.plot(kind = 'bar', x = 'mon_key_2', y = False, rot = 360, figsize = (10,5), legend = False, color=c)

    for p in ax.patches:
        width, height = p.get_width(), p.get_height()
        x, y = p.get_xy() 
        ax.text(x+width/2, y+height/2, '{:.0f}'.format(height), horizontalalignment='center', verticalalignment='center', color = 'white')
    
    ax2 = FL_MI_eRx_month_wise_line[:-1].plot(ax = ax, secondary_y=True, kind = 'line', x = 'mon_key_2', y = 'per_growth', color = 'grey', legend = False)

    texts = []
    
    for p,k in zip(ax.patches, FL_MI_eRx_month_wise_line.per_growth):
        width, height = p.get_width(), p.get_height()
        x, y = p.get_xy() 
        text = ax.text(x+width/2, y+height, str(round(k,2))+'%', horizontalalignment='center', verticalalignment='center')
        texts.append(text)
    adjust_text(texts)
    
    labels = [item.get_text() for item in ax.get_xticklabels()]
    labels[6] = 'Avg'

    ax.set_xticklabels(labels)

    
    ax.set_xlabel("Month Year",fontsize = 16,fontname = 'Arial')
    ax.set_ylabel('Count of Rx', fontsize = 16,fontname = 'Arial')
    ax2.set_ylabel('per_growth', fontsize = 16,fontname = 'Arial')
    #ax.set_ylim(0,1400)

    max_y_limit = FL_MI_eRx_month_wise_line[False].max()
    per_10 = (10/100) * max_y_limit + max_y_limit
    ax.set_ylim(0, per_10)

    plt.title(str(i)+" non eRx M-o-M analysis Trend",fontsize = 16,fontname = 'Arial')

    plt.legend(title='non eRx', bbox_to_anchor=(1.0, 1), loc='upper left')

    ax2.spines['top'].set_visible(False)
    ax2.spines['left'].set_visible(False)
    ax2.spines['right'].set_visible(False)
    ax2.spines['bottom'].set_visible(False)

    ax.spines['top'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    
    plt.savefig(output_image_path_billed+str(i)+'_billed_non_eRx_trend.png', bbox_inches='tight', dpi=300,pad_inches = 0)
    plt.close()

In [ ]:
# Delivery analysis

In [ ]:
# erx_delivery_data = erx_data[(erx_data['max_date'] > six_month_date)]
erx_delivery_data = erx_data
erx_delivery_data_pie_pivot = pd.pivot_table(dff, values = 'rx_code_key_location', index=['delivery_status'], aggfunc = 'count').reset_index()
# erx_delivery_data_pie_pivot.iloc[0,1]=32680
# erx_delivery_data_pie_pivot.iloc[1,1]=3632
mylabels = (erx_delivery_data_pie_pivot['rx_code_key_location']/6).to_list()
my_labels = 'Delivered '+str(f'{int(mylabels[0]):,}'),'pickedup '+str(f'{int(mylabels[1]):,}')
my_explode = (0, 0.1)
plt.pie(erx_delivery_data_pie_pivot['rx_code_key_location']/6, labels=my_labels, autopct='%1.2f%%', startangle=90, shadow = False, explode=my_explode)
plt.title('Mode of Rx Transfer Delivery')
plt.axis('equal')

plt.savefig(output_image_path_billed+'Mode_of_Rx_Transfer_Delivery.png', bbox_inches='tight', dpi=300,pad_inches = 0)
# plt.show()

plt.close()


In [ ]:
erx_delivery_data['pharmacy loc'].unique()

In [ ]:
# erx_delivery_data = erx_data[(erx_data['max_date'] > six_month_date)]

erx_delivery_data_month_wise = pd.pivot_table(dff, values = 'rx_code_key_location', columns = 'delivery_status', index = ['mon_key_2'], aggfunc = 'count').reset_index()

erx_delivery_data_month_wise['mon_key_2'] = pd.to_datetime(erx_delivery_data_month_wise['mon_key_2'], format = "%b-%y")
erx_delivery_data_month_wise['mon_key_3'] = erx_delivery_data_month_wise['mon_key_2'].dt.strftime('%b-%y')
erx_delivery_data_month_wise.sort_values(by = 'mon_key_2', inplace = True)
erx_delivery_data_month_wise.drop(columns = 'mon_key_2', inplace = True)
erx_delivery_data_month_wise.rename(columns = {'mon_key_3':'mon_key_2'}, inplace = True)

erx_delivery_data_month_wise

In [ ]:
erx_delivery_data_month_wise_line = erx_delivery_data_month_wise
erx_delivery_data_month_wise_line.columns
erx_delivery_data_month_wise_line['Total'] = erx_delivery_data_month_wise_line['Pickedup'] + erx_delivery_data_month_wise_line['Delivered']
erx_delivery_data_month_wise_line['per change'] = round(erx_delivery_data_month_wise_line['Total'].pct_change()*100,2)
erx_delivery_data_month_wise_line = erx_delivery_data_month_wise_line.fillna(0)
erx_delivery_data_month_wise_line

In [ ]:
# erx_delivery_data = erx_data[(erx_data['max_date'] > six_month_date)]
erx_delivery_data = erx_data
erx_delivery_data_month_wise = pd.pivot_table(dff, values = 'rx_code_key_location', columns = 'delivery_status', index = ['mon_key_2'], aggfunc = 'count').reset_index()

erx_delivery_data_month_wise['mon_key_2'] = pd.to_datetime(erx_delivery_data_month_wise['mon_key_2'], format = "%b-%y")
erx_delivery_data_month_wise['mon_key_3'] = erx_delivery_data_month_wise['mon_key_2'].dt.strftime('%b-%y')
erx_delivery_data_month_wise.sort_values(by = 'mon_key_2', inplace = True)
erx_delivery_data_month_wise.drop(columns = 'mon_key_2', inplace = True)
erx_delivery_data_month_wise.rename(columns = {'mon_key_3':'mon_key_2'}, inplace = True)

erx_delivery_data_month_wise

In [ ]:
ax = erx_delivery_data_month_wise.plot(kind = 'bar', stacked = True, x = 'mon_key_2', rot = 360, figsize = (10,5), legend = False)
erx_delivery_data_month_wise_line.plot(ax = ax, kind = 'line', x = 'mon_key_2', y = 'Total', color = 'grey', legend = False)

# # for p in ax.patches:
# #     b = p.get_bbox()
# #     val = "{}".format(int(p.get_height()))
# #     m = p.get_height()
# #     plt.annotate(val, ((b.x0 + b.x1)/2.05 , b.y1),fontsize=10, ha='center', va='center')

for p in ax.patches:
    width, height = p.get_width(), p.get_height()
    x, y = p.get_xy() 
    ax.text(x+width/2, y+height/2, f'{int(height):,}', horizontalalignment='center', verticalalignment='center')

rects = ax.patches

texts = []

for rect, total, per in zip(rects, erx_delivery_data_month_wise_line['Total'], erx_delivery_data_month_wise_line['per change']):
    text = ax.text(rect.get_x() + rect.get_width() / 2, total, f'{int(total):,}'+' ('+str(per)+'% '+')',#str(per)+'% '+f'{int(total):,}',
            ha='center', va='center')
    texts.append(text)
adjust_text(texts)
    
plt.ylabel("Count of Rx",fontsize = 16,fontname = 'Arial')
plt.xlabel("Month Year",fontsize = 16,fontname = 'Arial')
plt.title("Rx Delivery M-o-M analysis",fontsize = 16,fontname = 'Arial')

max_y_limit = erx_delivery_data_month_wise_line['Total'].max()
per_10 = (10/100) * max_y_limit + max_y_limit
ax.set_ylim(0, per_10)

orange_patch = mpatches.Patch(color = 'orange', label = 'Pickedup')
blue_patch = mpatches.Patch(color = 'steelblue', label = 'Delivered')
trend = mpatches.Patch(color = 'grey', label = 'Total')

plt.legend(handles = [orange_patch, blue_patch, trend],title='is pickedup?', bbox_to_anchor=(1.0, 1), loc='upper left')

ax.spines['top'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)

plt.savefig(output_image_path_billed+'Delivery_M-o-M_analysis_eRx.png', bbox_inches='tight', dpi=300,pad_inches = 0)
plt.close()
# plt.show()

In [ ]:
erx_delivery_data_pharmacy_wise = pd.pivot_table(dff, values = 'rx_code_key_location', columns = 'delivery_status', index = ['pharmacy loc'], aggfunc = 'count').reset_index()
erx_delivery_data_pharmacy_wise.rename(columns = {False: 'False', True: 'True'}, inplace =True)
print(erx_delivery_data_pharmacy_wise)
df_total = erx_delivery_data_pharmacy_wise['Delivered'] + erx_delivery_data_pharmacy_wise['Pickedup']
erx_delivery_data_pharmacy_wise['Delivered'] = erx_delivery_data_pharmacy_wise[erx_delivery_data_pharmacy_wise.columns[1:2]].div(df_total, 0)*100
erx_delivery_data_pharmacy_wise['Pickedup'] = erx_delivery_data_pharmacy_wise[erx_delivery_data_pharmacy_wise.columns[2:]].div(df_total, 0)*100

ax = erx_delivery_data_pharmacy_wise.plot(kind = 'bar', stacked = True, x = 'pharmacy loc', rot = 360)

for p in ax.patches:
    width, height = p.get_width(), p.get_height()
    x, y = p.get_xy() 
    ax.text(x+width/2, y+height/2, '{:.0f}%'.format(height), horizontalalignment='center', verticalalignment='center')

plt.ylabel("Percentage Rx",fontsize = 16,fontname = 'Arial')
plt.xlabel("Pharmacy Location",fontsize = 16,fontname = 'Arial')
plt.title("Location wise mode of Delivery",fontsize = 16,fontname = 'Arial')

orange_patch = mpatches.Patch(color = 'orange', label = 'Pickedup')
blue_patch = mpatches.Patch(color = 'steelblue', label = 'Delivered')

plt.legend(handles = [orange_patch, blue_patch],title='is pickedup?', bbox_to_anchor=(1.0, 1), loc='upper left')

ax.spines['top'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
#erx_delivery_data_pharmacy_wise

plt.savefig(output_image_path_billed+'Location_wise_mode_of_delivery.png', bbox_inches='tight', dpi=300,pad_inches = 0)
plt.close()
# plt.show()

In [ ]:
len(erx_delivery_data)

In [ ]:
erx_delivery_data_pharmacy_wise_Rx_tbl = pd.pivot_table(dff, values = 'rx_code_key_location', columns = 'pharmacy loc', index = ['delivery_status'], aggfunc = 'count').reset_index()
#erx_delivery_data_pharmacy_wise_Rx_tbl = erx_delivery_data_pharmacy_wise_Rx_tbl.rename(columns = {'pharmacy_loc': 'pharmacy loc'})
erx_delivery_data_pharmacy_wise_Rx_tbl = erx_delivery_data_pharmacy_wise_Rx_tbl.append(erx_delivery_data_pharmacy_wise_Rx_tbl.sum(axis=0), ignore_index = True)

erx_delivery_data_pharmacy_wise_Rx_tbl = erx_delivery_data_pharmacy_wise_Rx_tbl.rename(columns = {'delivery_status': 'pharmacy loc/is pickedup'})
erx_delivery_data_pharmacy_wise_Rx_tbl.iloc[[0],[0]] = 'Delivered'
erx_delivery_data_pharmacy_wise_Rx_tbl.iloc[[1],[0]] = 'Pickedup'
erx_delivery_data_pharmacy_wise_Rx_tbl.iloc[[2],[0]] = 'Total'
erx_delivery_data_pharmacy_wise_Rx_tbl
erx_delivery_data_pharmacy_wise_Rx_tbl.set_index('pharmacy loc/is pickedup', inplace = True)
erx_delivery_data_pharmacy_wise_Rx_tbl['Total'] = erx_delivery_data_pharmacy_wise_Rx_tbl.sum(axis=1)
erx_delivery_data_pharmacy_wise_Rx_tbl = erx_delivery_data_pharmacy_wise_Rx_tbl.fillna(0)
erx_delivery_data_pharmacy_wise_Rx_tbl = erx_delivery_data_pharmacy_wise_Rx_tbl[erx_delivery_data_pharmacy_wise_Rx_tbl.columns[:]].div(6,0).astype(int)
erx_delivery_data_pharmacy_wise_Rx_tbl = erx_delivery_data_pharmacy_wise_Rx_tbl.sort_values(erx_delivery_data_pharmacy_wise_Rx_tbl.last_valid_index(), axis = 1, ascending = False)
erx_delivery_data_pharmacy_wise_Rx_tbl
subset_value = (erx_delivery_data_pharmacy_wise_Rx_tbl.index.to_list()[:-1], 
               ['NY-BK', 'NJ-SO', 'PA-PH', 'FL-MI', 'NY-MA', 'MD-BA', 'NC-RA'])

erx_delivery_data_pharmacy_wise_Rx_tbl = erx_delivery_data_pharmacy_wise_Rx_tbl[['NY-BK', 'NJ-SO', 'PA-PH', 'FL-MI', 'NY-MA', 'MD-BA', 'NC-RA','Total']]

df_style = erx_delivery_data_pharmacy_wise_Rx_tbl.style.set_properties(**{'text-align': 'center','border-color':'white','border-width':'thin','border-style':'solid'}).background_gradient(cmap='GnBu',axis=1, subset = subset_value).format("{:,}")

#df_style = erx_delivery_data_pharmacy_wise_Rx_tbl.style.background_gradient(cmap ='GnBu', axis = 1).format("{:,}")
dfi.export(df_style, output_image_path_billed+'erx_delivery_data_pharmacy_wise_Rx_count_table.png')
df_style

In [ ]:
# erx_delivery_data

In [ ]:
# Rx as pickedup

In [ ]:
erx_delivery_data['pharmacy loc'].unique()

In [ ]:
for i in np.unique(erx_delivery_data['pharmacy loc']):
    delivery_df = dff[dff['pharmacy loc'] == i]

    D_pickedup = delivery_df[delivery_df['is_pickedup'] == True]
    #FL_MI_d_delivered = delivery_df[delivery_df['is_pickedup'] == False]

    delivery_Rx_month_wise = pd.pivot_table(D_pickedup, values = 'rx_code_key_location', columns = 'delivery_status', index = ['mon_key_2'], aggfunc = 'count').reset_index()
#     print(delivery_Rx_month_wise)
    delivery_Rx_month_wise['mon_key_2'] = pd.to_datetime(delivery_Rx_month_wise['mon_key_2'], format = "%b-%y")
    delivery_Rx_month_wise['mon_key_3'] = delivery_Rx_month_wise['mon_key_2'].dt.strftime('%b-%y')
    delivery_Rx_month_wise.sort_values(by = 'mon_key_2', inplace = True)
    delivery_Rx_month_wise.drop(columns = 'mon_key_2', inplace = True)
    delivery_Rx_month_wise.rename(columns = {'mon_key_3': 'mon_key_2'}, inplace =True)
    delivery_Rx_month_wise['per_growth'] = round(delivery_Rx_month_wise['Pickedup'].pct_change()*100,2)
#     print(delivery_Rx_month_wise)
    avgdf = pd.DataFrame(columns = ['Pickedup', 'mon_key_2'], index=range(0,1))
    avgdf['Pickedup'] = int(delivery_Rx_month_wise['Pickedup'].mean())
    avgdf['mon_key_2'] = 'avg'

    delivery_Rx_month_wise = delivery_Rx_month_wise.append(avgdf)
    delivery_Rx_month_wise = delivery_Rx_month_wise.fillna(0)
    delivery_Rx_month_wise_line = delivery_Rx_month_wise
    
    if len(delivery_Rx_month_wise) == 5:
        c = ['C0', 'C0', 'C0', 'C0', 'green']
    elif len(delivery_Rx_month_wise) == 6:
        c = ['C0', 'C0', 'C0', 'C0','C0', 'green']
    else:
        c = ['C0', 'C0', 'C0', 'C0', 'C0','C0', 'green']
        
    ax = delivery_Rx_month_wise.plot(kind = 'bar', x = 'mon_key_2', y = 'Pickedup', rot = 360, figsize = (10,5), legend = False, color=c)
    
    for p in ax.patches:
        width, height = p.get_width(), p.get_height()
        x, y = p.get_xy() 
        ax.text(x+width/2, y+height/2, '{:.0f}'.format(height), horizontalalignment='center', verticalalignment='center', color = 'white')
    
    ax2 = delivery_Rx_month_wise_line[:-1].plot(ax = ax, secondary_y=True, kind = 'line', x = 'mon_key_2', y = 'per_growth', color = 'grey', legend = False)
    texts = []
    for p,k in zip(ax.patches, delivery_Rx_month_wise_line.per_growth):
        width, height = p.get_width(), p.get_height()
        x, y = p.get_xy() 
        text = ax.text(x+width/2, y+height, str(round(k,2))+'%', horizontalalignment='center', verticalalignment='center')
        texts.append(text)
    adjust_text(texts)
    
    labels = [item.get_text() for item in ax.get_xticklabels()]
    labels[6] = 'Avg'

    ax.set_xticklabels(labels)
    
    ax.set_xlabel('Month Year', fontsize = 16,fontname = 'Arial')
    ax.set_ylabel('Count of Rx', fontsize = 16,fontname = 'Arial')
    ax2.set_ylabel('per_growth', fontsize = 16,fontname = 'Arial')
    #ax.set_ylim(0,1400)
    max_y_limit = delivery_Rx_month_wise_line['Pickedup'].max()
    per_10 = (10/100) * max_y_limit + max_y_limit
    ax.set_ylim(0, per_10)
    
    plt.xlabel("Month Year",fontsize = 16,fontname = 'Arial')
    plt.title(str(i)+" Rx pickedup M-o-M analysis",fontsize = 16,fontname = 'Arial')

    ax.spines['top'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)

    ax2.spines['top'].set_visible(False)
    ax2.spines['left'].set_visible(False)
    ax2.spines['right'].set_visible(False)
    ax2.spines['bottom'].set_visible(False)
    
    plt.legend(title='Pickedup', bbox_to_anchor=(1.0, 1), loc='upper left')
    #delivery_Rx_month_wise

    plt.savefig(output_image_path_billed+str(i)+'_pickedup_trend.png', bbox_inches='tight', dpi=300,pad_inches = 0)
    plt.close()
#     plt.show()

In [ ]:
# Rx as delivered

In [ ]:
for i in np.unique(erx_delivery_data['pharmacy loc']):
    delivery_df = dff[dff['pharmacy loc'] == i]

    #D_pickedup = delivery_df[delivery_df['is_pickedup'] == True]
    D_delivered = delivery_df[delivery_df['is_pickedup'] == False]

    delivery_Rx_month_wise = pd.pivot_table(D_delivered, values = 'rx_code_key_location', columns = 'delivery_status', index = ['mon_key_2'], aggfunc = 'count').reset_index()
    delivery_Rx_month_wise['mon_key_2'] = pd.to_datetime(delivery_Rx_month_wise['mon_key_2'], format = "%b-%y")
    delivery_Rx_month_wise['mon_key_3'] = delivery_Rx_month_wise['mon_key_2'].dt.strftime('%b-%y')
    delivery_Rx_month_wise.sort_values(by = 'mon_key_2', inplace = True)
    delivery_Rx_month_wise.drop(columns = 'mon_key_2', inplace = True)
    delivery_Rx_month_wise.rename(columns = {'mon_key_3': 'mon_key_2'}, inplace =True)
#     print(delivery_Rx_month_wise)
    delivery_Rx_month_wise['per_growth'] = round(delivery_Rx_month_wise['Delivered'].pct_change()*100,2)

    avgdf = pd.DataFrame(columns = ['Delivered', 'mon_key_2'], index=range(0,1))
    avgdf['Delivered'] = int(delivery_Rx_month_wise['Delivered'].mean())
    avgdf['mon_key_2'] = 'avg'

    delivery_Rx_month_wise = delivery_Rx_month_wise.append(avgdf)
    delivery_Rx_month_wise = delivery_Rx_month_wise.fillna(0)
    delivery_Rx_month_wise_line = delivery_Rx_month_wise
    
    if len(delivery_Rx_month_wise_line) == 5:
        c = ['C0', 'C0', 'C0', 'C0', 'green']
    elif len(delivery_Rx_month_wise_line) == 6:
        c = ['C0', 'C0', 'C0', 'C0','C0', 'green']
    else:
        c = ['C0', 'C0', 'C0', 'C0', 'C0','C0', 'green']

    ax = delivery_Rx_month_wise.plot(kind = 'bar', x = 'mon_key_2', y = 'Delivered', rot = 360, figsize = (10,5), legend = False, color=c)

    for p in ax.patches:
        width, height = p.get_width(), p.get_height()
        x, y = p.get_xy() 
        ax.text(x+width/2, y+height/2, '{:.0f}'.format(height), horizontalalignment='center', verticalalignment='center', color = 'white')
    
    ax2 = delivery_Rx_month_wise_line[:-1].plot(ax = ax, secondary_y=True, kind = 'line', x = 'mon_key_2', y = 'per_growth', color = 'grey', legend = False)
    
    texts = []
    
    for p,k in zip(ax.patches, delivery_Rx_month_wise_line.per_growth):
        width, height = p.get_width(), p.get_height()
        x, y = p.get_xy() 
        text = ax.text(x+width/2, y+height, str(round(k,2))+'%', horizontalalignment='center', verticalalignment='center')
        texts.append(text)
    adjust_text(texts)
    
    labels = [item.get_text() for item in ax.get_xticklabels()]
    labels[6] = 'Avg'

    ax.set_xticklabels(labels)
    
    ax.set_xlabel('Month Year', fontsize = 16,fontname = 'Arial')
    ax.set_ylabel('Count of Rx', fontsize = 16,fontname = 'Arial')
    ax2.set_ylabel('per_growth', fontsize = 16,fontname = 'Arial')
    #ax.set_ylim(0,1400)
    max_y_limit = delivery_Rx_month_wise_line['Delivered'].max()
    per_10 = (10/100) * max_y_limit + max_y_limit
    ax.set_ylim(0, per_10)
    
    plt.xlabel("Month Year",fontsize = 16,fontname = 'Arial')
    plt.title(str(i)+" Rx delivered M-o-M analysis",fontsize = 16,fontname = 'Arial')

    ax.spines['top'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)

    ax2.spines['top'].set_visible(False)
    ax2.spines['left'].set_visible(False)
    ax2.spines['right'].set_visible(False)
    ax2.spines['bottom'].set_visible(False)
    
    plt.legend(title='Delivered', bbox_to_anchor=(1.0, 1), loc='upper left')
    #delivery_Rx_month_wise

    plt.savefig(output_image_path_billed+str(i)+'_delivered_trend.png', bbox_inches='tight', dpi=300,pad_inches = 0)
    plt.close()
# plt.show()

### Automation

In [ ]:
import pptx
import pandas as pd
import numpy as np
import glob
from pptx.enum.shapes import MSO_SHAPE_TYPE
from pptx.util import Inches
import copy
from pptx.shapes.autoshape import Shape

from pptx.shapes.graphfrm import GraphicFrame

from pptx.dml.color import RGBColor
from pptx.enum.dml import MSO_THEME_COLOR

from pptx.util import Inches,Pt
from pptx.enum.text import PP_ALIGN
from pptx.enum.dml import MSO_LINE
from pptx.enum.text import MSO_ANCHOR
from pptx.enum.dml import MSO_COLOR_TYPE, MSO_FILL
import re

In [ ]:
%run path_changer_refill.py

In [ ]:
_EMUS_PER_INCH = 914400
_EMUS_PER_CENTIPOINT = 127
_EMUS_PER_CM = 360000
_EMUS_PER_MM = 36000
_EMUS_PER_PT = 12700

In [ ]:
ppt_output = pptx.Presentation()

In [ ]:
wbr_template = pptx.Presentation('template_refill.pptx')

In [ ]:
ppt_output.slide_width = wbr_template.slide_width
ppt_output.slide_height = wbr_template.slide_height

In [ ]:
image_replace_path = output_image_path_billed

In [ ]:
""" create base template file """
writer = pd.ExcelWriter('excel.xlsx', engine='xlsxwriter')

for slide in wbr_template.slides:
    slide_no = (wbr_template.slides.index(slide))+1
    sheetname = "slide_{:02d}".format(slide_no)
    tdf = pd.DataFrame()
    
    for shape in slide.shapes:
        odf = pd.DataFrame([['','','','','','']],\
                           columns=['slide_no','shape_name','shape_type','shape_id','template_value','replace_value'])
        #odf = pd.DataFrame([])
        odf['slide_no'] = sheetname
        odf['shape_name'] = shape.name
        odf['shape_id'] = shape.shape_id
        
        if shape.shape_type == MSO_SHAPE_TYPE.PICTURE:
            odf['shape_type'] ='image'
            try:
                odf['template_value'] = copy.deepcopy(shape).element.xpath("//p:nvPicPr//p:cNvPr/@descr")[0] 
            except:
                odf['template_value'] = shape.name
        if shape.shape_type == MSO_SHAPE_TYPE.TEXT_BOX:
            odf['shape_type'] = 'text box'
            odf['template_value'] = shape.text
        
        tdf = tdf.append(odf)
    
    tdf.to_excel(writer,sheet_name=sheetname,index=False)

writer.save()

In [ ]:
schemecolormap = {'accent1': MSO_THEME_COLOR.ACCENT_1,\
                 'accent2': MSO_THEME_COLOR.ACCENT_2,\
                  'accent3': MSO_THEME_COLOR.ACCENT_3,\
                  'accent4': MSO_THEME_COLOR.ACCENT_4,\
                  'accent5': MSO_THEME_COLOR.ACCENT_5,\
                  'accent6': MSO_THEME_COLOR.ACCENT_6,\
                  'bg1': MSO_THEME_COLOR.BACKGROUND_1,\
                  'bg2': MSO_THEME_COLOR.BACKGROUND_2,\
                  'dk1': MSO_THEME_COLOR.DARK_1,\
                  'dk2': MSO_THEME_COLOR.DARK_2,\
                  'lt1': MSO_THEME_COLOR.LIGHT_1,\
                  'lt2': MSO_THEME_COLOR.LIGHT_2,\
                  'tx1': MSO_THEME_COLOR.TEXT_1,\
                  'tx2': MSO_THEME_COLOR.TEXT_2
                 
                 }

In [ ]:
paragraph_align_map = {
    "ctr": PP_ALIGN.CENTER,\
    "r": PP_ALIGN.RIGHT,
    "l": PP_ALIGN.LEFT,
    
}

In [ ]:
box_border_dashmap = {
    
    "solid": MSO_LINE.SOLID,\
    "dot": MSO_LINE.ROUND_DOT,\
    "dash": MSO_LINE.DASH,\
    "lgDash": MSO_LINE.LONG_DASH,\
    "dashDot": MSO_LINE.DASH_DOT,\
    "lgDashDot": MSO_LINE.LONG_DASH_DOT,\
    "lgDashDotDot": MSO_LINE.DASH_DOT_DOT,\
    "sysDash": MSO_LINE.DASH,\
    "sysDot": MSO_LINE.SQUARE_DOT,\
    "sysDashDot": MSO_LINE.DASH_DOT,\
    "sysDashDotDot": MSO_LINE.DASH_DOT_DOT,\
    
    
}

In [ ]:
def get_rgb_color(color_string):

    rcol = int(color_string[0:2],16)
    bcol = int(color_string[2:4],16)
    gcol = int(color_string[4:6],16)
    
    return RGBColor(rcol,bcol,gcol)

In [ ]:
def get_textbox_properties(sph):
    #print(sph.element.xml)
    
    fillcolor_schemecolor = (sph.element.xpath("//p:spPr/a:solidFill//a:schemeClr/@val"))
    fillcolor_rbgcolor = (sph.element.xpath("//p:spPr/a:solidFill//a:srgbClr/@val"))
    fontsz = (sph.element.xpath("//a:rPr/@sz"))
    fontface = (sph.element.xpath("//a:rPr//@typeface"))
    fillbrightness = sph.element.xpath("//p:spPr/a:solidFill//a:schemeClr//a:lumOff//@val")
    filldarkness = sph.element.xpath("//p:spPr/a:solidFill//a:schemeClr//a:lumMod//@val")
    
    font_isbold = (sph.element.xpath("//a:rPr/@b"))
    font_isitalic = (sph.element.xpath("//a:rPr/@i"))
    font_isunderline = (sph.element.xpath("//a:rPr/@u"))
    font_iscentered = (sph.element.xpath("//a:rPr/a:cs"))
    font_alignment = (sph.element.xpath("//p:txBody//a:p//a:pPr/@algn"))
    
    box_border_schemecolor = (sph.element.xpath("//p:spPr//a:ln//a:solidFill//a:schemeClr/@val"))
    box_border_rbgcolor = (sph.element.xpath("//p:spPr//a:ln//a:solidFill//a:srgbClr/@val"))
    box_border_width = (sph.element.xpath("//p:spPr//a:ln/@w"))
    box_border_brightness = (sph.element.xpath("//p:spPr//a:ln//a:solidFill//a:schemeClr//a:lumOff/@val"))
    box_border_darkness = (sph.element.xpath("//p:spPr//a:ln//a:solidFill//a:schemeClr//a:lumMod/@val"))
    box_border_isdash = (sph.element.xpath("//p:spPr//a:ln//a:prstDash/@val"))
    
    
    if (len(fillbrightness)==0)&(len(filldarkness)!=0):
        fillbrightness = [100000-int(filldarkness[0])]
    
    if (len(box_border_brightness)==0)&(len(box_border_darkness)!=0):
        box_border_brightness = [100000-int(box_border_darkness[0])]
       
    if len(font_iscentered)!=0:
        palign = paragraph_align_map.get("ctr")
    else:
        try:
            palign = paragraph_align_map.get(font_alignment[0])
        except:
            palign = paragraph_align_map.get('r')
    
    #print(sph.fill.fore_color.type==MSO_COLOR_TYPE.SCHEME)
    
    outdict = {
        
        #"fillcolor_schemecolor": schemecolormap.get(fillcolor_schemecolor[0]) if len(fillcolor_schemecolor)!=0 else None,\
        "fillcolor_schemecolor": (sph.fill.fore_color.theme_color) if len(fillcolor_schemecolor)!=0 else None,\
        "fillcolor_rbgcolor": get_rgb_color(fillcolor_rbgcolor[0]) if len(fillcolor_rbgcolor)!=0 else None,\
        "fontsz": (int(fontsz[0])/100) if len(fontsz)!=0 else None,\
        "fontface": fontface[0] if len(fontface)!=0 else None,\
        "fillbrightness": (int(fillbrightness[0])/100000) if len(fillbrightness)!=0 else None,\
        "font_isbold": True if len(font_isbold)!=0 else False,\
        "font_isitalic": True if len(font_isitalic)!=0 else False,\
        "font_isunderline": True if len(font_isunderline)!=0 else False,\
        "font_alignment": palign,\
        
        "box_border_schemecolor": schemecolormap.get(box_border_schemecolor[0]) if len(box_border_schemecolor)!=0 else None,\
        "box_border_rbgcolor": get_rgb_color(box_border_rbgcolor[0]) if len(box_border_rbgcolor)!=0 else None,\
        "box_border_brightness": (int(box_border_brightness[0])/100000) if len(box_border_brightness)!=0 else None,\
        "box_border_width" : (int(box_border_width[0])/_EMUS_PER_PT) if len(box_border_width)!=0 else None,\
        "box_border_isdash":box_border_dashmap.get(box_border_isdash[0]) if len(box_border_isdash)!=0 else None
        
        
    }
    
    return outdict
    

In [ ]:
def set_textbox_properties(tb,prop_dict):
    print(prop_dict)
    tf = tb.text_frame
    p = tf.paragraphs[0]
    run =p.add_run()
    #run.text = shape.text
    
    if prop_dict["fillcolor_schemecolor"]:
        tb.fill.solid()
        print(prop_dict["fillcolor_schemecolor"])
        tb.fill.fore_color.theme_color = prop_dict["fillcolor_schemecolor"]
    
    if prop_dict["fillcolor_rbgcolor"]:
        tb.fill.solid()
        tb.fill.fore_color.rgb = prop_dict["fillcolor_rbgcolor"]
        
    if prop_dict["fillbrightness"]:
        tb.fill.fore_color.brightness = prop_dict["fillbrightness"]
        #pass
    
    if prop_dict["fontsz"]:
        run.font.size=Pt(prop_dict["fontsz"])
        
    if prop_dict["fontface"]:
        run.font.name =prop_dict["fontface"]
    
    run.font.bold = prop_dict["font_isbold"]
    run.font.italic = prop_dict["font_isitalic"]
    run.font.underline = prop_dict["font_isunderline"]
    
    p.alignment = prop_dict["font_alignment"]
    
    if prop_dict["box_border_schemecolor"]:
        tb.line.color.theme_color = prop_dict["box_border_schemecolor"]
        
    if prop_dict["box_border_rbgcolor"]:
        tb.line.color.rgb = prop_dict["box_border_rbgcolor"]
    
    if prop_dict["box_border_brightness"]:
        #pass
        tb.line.color.brightness = prop_dict["box_border_brightness"]
    
    if prop_dict["box_border_width"]:
        tb.line.width = Pt(prop_dict["box_border_width"])
        
    if prop_dict["box_border_isdash"]:
        tb.line.dash_style = prop_dict["box_border_isdash"]
    
    
        
    return run

In [ ]:
def set_text_holders(stext,text_replace_dict):
    otext = str(stext)
    sgr = (re.findall(r'{{(.*?)}}',otext))
       
    for sl in sgr:
        print(sl)
        print(str(text_replace_dict.get(sl)))
        otext = (otext).replace(sl,str(text_replace_dict.get(sl)))\
            .replace("{","").replace("}","")
    
    return otext   

In [ ]:
total_revenue = 7.17
rx_count = 234
active_provider = 100
patient_served = 678
new_provider = 897
rev_per_rx = 23

In [ ]:
def get_variable_names(stext):
    otext = str(stext)
    sgr = (re.findall(r'{{(.*?)}}',otext))
    return sgr

In [ ]:
def create_image_dict(df):
    xdf = df[['shape_name','replace_value']]
    return xdf.set_index('shape_name').to_dict()["replace_value"]

In [ ]:
def create_variable_dict(df):
    xdf = df[['template_value']]
    
    hlist = []
    for i,r in xdf.iterrows():
        
        xlist = get_variable_names(r['template_value'])
        hlist = hlist + xlist
    
    vdict = {}
    for x in np.unique(np.array(hlist)):
        vdict[x] = eval(x)
    return vdict
    

In [ ]:
variable_template_df = pd.ExcelFile('Final_excel.xlsx')

In [ ]:
for slide in wbr_template.slides:
    slide_no = (wbr_template.slides.index(slide))+1
    sheetname = "slide_{:02d}".format(slide_no)
    tslide = slide
    #print(tslide)
    oslide = ppt_output.slides.add_slide(ppt_output.slide_layouts[6])
    vtdf = variable_template_df.parse(sheetname)
    image_replace_dict = create_image_dict(vtdf[vtdf['shape_type']=='image'])
    text_replace_dict = create_variable_dict(vtdf[vtdf['shape_type']=='text box'])
    print(image_replace_dict)
    for shape in tslide.shapes:
        #print(shape._element.xml)
        #print(shape.name)
        sleft = Inches(shape.left/_EMUS_PER_INCH)
        swidth = Inches(shape.width/_EMUS_PER_INCH)
        sheight = Inches(shape.height/_EMUS_PER_INCH)
        stop = Inches(shape.top/_EMUS_PER_INCH)

        if shape.shape_type == MSO_SHAPE_TYPE.PICTURE:
            try:
                img_path = image_replace_path+image_replace_dict.get(shape.name)
                oslide.shapes.add_picture(img_path,left=sleft,top=stop,width=swidth,height=sheight)
            except:
                pass


        if shape.shape_type == MSO_SHAPE_TYPE.TEXT_BOX:
            #print(shape.shape_id)

            cshape = copy.deepcopy(shape)
            property_dict = (get_textbox_properties(cshape))

            tb = oslide.shapes.add_textbox(left=sleft,top=stop,width=swidth,height=sheight)
            run = set_textbox_properties(tb,property_dict)

            run.text = (set_text_holders(shape.text,text_replace_dict))
            print(run.text)

    ppt_output.save('Sept_Slide_1_13_refill.pptx')
       

In [ ]:
oslide.shapes[0].element.xml

In [ ]:
#20/09/21